In [53]:
import shap
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.inspection import permutation_importance
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import f_regression, chi2
from scipy.stats import ttest_ind
from sklearn.metrics import mean_squared_log_error,mean_absolute_error,mean_squared_error,r2_score

from scipy.stats import zscore, skew

import sklearn
sklearn.set_config(transform_output="pandas")

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, OrdinalEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier,RandomForestRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import optuna

In [55]:
pd.set_option('display.max_columns', None)

In [57]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

submission = pd.read_csv('./sample_submission.csv')

In [59]:
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,

In [92]:
train.info(),train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

(None,
                 Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
 count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
 mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
 std     421.610009    42.300571    24.284752    9981.264932     1.382997   
 min       1.000000    20.000000    21.000000    1300.000000     1.000000   
 25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
 50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
 75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
 max    1460.000000   190.000000   313.000000  215245.000000    10.000000   
 
        OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  \
 count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000   
 mean      5.575342  1971.267808   1984.865753   103.685262   443.639726   
 std       1.112799    30.202904     20.645407   181.066207   456.0980

In [61]:
Mean = ['LotFrontage','MasVnrArea','GarageYrBlt']
Moda = ['Electrical','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
       'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond']
num_fill_zero = ['Alley']
d_f = ['PoolQC','PoolArea','MiscFeature','Fence','MasVnrType']

my_imputer = ColumnTransformer(
    transformers = [
        ('drop_feature','drop',d_f),
        ('num_zero', SimpleImputer(strategy='constant', fill_value=0), num_fill_zero),
        ('num_i', SimpleImputer(strategy='mean'),Mean),
        ('cat_i', SimpleImputer(strategy='most_frequent'),Moda)
    ],
    verbose_feature_names_out = False,
    remainder = 'passthrough'
)

train = my_imputer.fit_transform(train)

In [63]:
# corr = train[['GarageFinish','GarageQual','GarageCond','SalePrice']].corr()
# sns.heatmap(corr,annot=True,cmap="coolwarm")

In [370]:
feature = pd.DataFrame(train['Age'])
target = pd.DataFrame(train['SalePrice'])

f_scores, p_values = f_regression(ss,d)

print("p-value", p_values)
print("f_scores:", f_scores)

p-value [0.26941319]
f_scores: [1.22066133]


D:\condaaa\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [65]:
# train_pca = PCA()
# sk_X_transformed = sk_pca.fit_transform(X)

In [67]:
train['TotalBathrooms'] = (train['FullBath'] + train['HalfBath']*0.5 + train['BsmtFullBath'] + train['BsmtHalfBath']*0.5) 
train['TotalPorchSF'] = (train['OpenPorchSF'] + train['EnclosedPorch'] + train['3SsnPorch'] +train['ScreenPorch'])
# train['HasBasement'] = (train['TotalBsmtSF']>0).astype(int)
# train['HasGarage'] = (train['GarageArea']>0).astype(int)
train['TotalLivArea'] = train['GrLivArea']+train['BsmtFinSF1']+train['BsmtFinSF2']
# train['IsMultiFloor'] = (train['2ndFlrSF']>0).astype(int)
# train['HasFinishedBsmt'] = ((train['BsmtFinSF1'] + train['BsmtFinSF2'])>0).astype(int)
train['TotalRooms'] = train['TotRmsAbvGrd'] + train['BedroomAbvGr']
# train['IsNew'] = (train['YearBuilt']>2000).astype(int)
train['Age'] = train['YrSold'] - train['YearBuilt']

train = train.drop(['FullBath','HalfBath','BsmtFullBath','BsmtHalfBath','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','GrLivArea','BsmtFinSF1','BsmtFinSF2','TotRmsAbvGrd','BedroomAbvGr','YrSold','YearBuilt'],axis=1)

In [69]:
train = train.drop(['GarageCond',],axis=1)

In [71]:
# for col in num_f:
#     if train[col].nunique() > 10:
#         sk = skew(train[col].dropna())
#         if abs(sk) > 0.75:
#             print(f"{col}: skew = {sk:.2f}")

In [73]:
train['Alley'] = train['Alley'].replace({'Grvl': 1})
train['Alley'] = train['Alley'].replace({'Pave': 1})

C:\Users\user\AppData\Local\Temp\ipykernel_1900\197578857.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Alley'] = train['Alley'].replace({'Pave': 1})


In [75]:
num_f = train.select_dtypes(include=['int64','float64'])
cat_f = train.select_dtypes(include=['object'])

In [77]:
s=cat_f.columns.tolist()

In [79]:
# train['GarageQual'].value_counts()

In [81]:
x,y = train.drop(['SalePrice','Id'],axis=1),train['SalePrice']

X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

In [83]:
model = CatBoostRegressor(loss_function='RMSE',eval_metric='MSLE',cat_features=s,random_seed=42)
model.fit(X_train, y_train)

Learning rate set to 0.04196
0:	learn: 0.1501228	total: 70.3ms	remaining: 1m 10s
1:	learn: 0.1428639	total: 132ms	remaining: 1m 5s
2:	learn: 0.1353796	total: 200ms	remaining: 1m 6s
3:	learn: 0.1284516	total: 265ms	remaining: 1m 5s
4:	learn: 0.1224519	total: 324ms	remaining: 1m 4s
5:	learn: 0.1164199	total: 389ms	remaining: 1m 4s
6:	learn: 0.1110935	total: 448ms	remaining: 1m 3s
7:	learn: 0.1060615	total: 513ms	remaining: 1m 3s
8:	learn: 0.1018397	total: 571ms	remaining: 1m 2s
9:	learn: 0.0975034	total: 635ms	remaining: 1m 2s
10:	learn: 0.0934030	total: 693ms	remaining: 1m 2s
11:	learn: 0.0898455	total: 758ms	remaining: 1m 2s
12:	learn: 0.0863249	total: 818ms	remaining: 1m 2s
13:	learn: 0.0827569	total: 880ms	remaining: 1m 1s
14:	learn: 0.0795106	total: 940ms	remaining: 1m 1s
15:	learn: 0.0764175	total: 997ms	remaining: 1m 1s
16:	learn: 0.0732925	total: 1.06s	remaining: 1m 1s
17:	learn: 0.0708522	total: 1.12s	remaining: 1m 1s
18:	learn: 0.0684038	total: 1.19s	remaining: 1m 1s
19:	learn:

In [87]:
train_pred = model.predict(X_train)
train_rmsle_score = np.sqrt(mean_squared_log_error(y_train,train_pred))
train_r2_score = r2_score(y_train, train_pred)

print(f"RMSLE train: {train_rmsle_score:.4f}")
print(f"R2 train: {train_r2_score:.4f}")

valid_pred

valid_pred = model.predict(X_valid)
valid_rmsle_score = np.sqrt(mean_squared_log_error(y_valid,valid_pred))
valid_r2_score = r2_score(y_valid, valid_pred)

print(f"RMSLE valid: {valid_rmsle_score:.4f}")
print(f"R2 valid: {valid_r2_score:.4f}")

RMSLE train: 0.0656
R2 train: 0.9825
RMSLE valid: 0.1399
R2 valid: 0.9069
